In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [3]:
#загружаю данные
df = pd.read_csv("C:/Users/Home/Desktop/семинар наставника 2 семестр/дз1 до 05.03/task_3.csv")
df.head()

,Unnamed: 0,NG_1,NG_2,NG_3,NG_4,NG_5,NG_6,NG_7,NG_8,NG_9,...,XZ_7,XZ_8,XZ_9,Kingdom,Phylum,Class,Order,Family,Genus,Species
0,ASV1,0.002905,0.005164,0.009326,0.004054,0.005028,0.002684,0.001880,0.000499,0.000921,...,0.000000,0.000621,0.000770,Bacteria,Firmicutes,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,NaN
1,ASV2,0.000000,0.001291,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001081,0.001474,0.000000,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,NaN,NaN,NaN
2,ASV3,0.009913,0.002582,0.010048,0.007842,0.008782,0.001342,0.016650,0.007814,0.006531,...,0.014296,0.012414,0.007184,Bacteria,Actinobacteriota,Thermoleophilia,Gaiellales,Gaiellaceae,Gaiella,NaN
3,ASV4,0.015724,0.008284,0.016193,0.009237,0.027955,0.027396,0.028734,0.024771,0.036172,...,0.030671,0.031267,0.023092,Bacteria,Actinobacteriota,Thermoleophilia,Solirubrobacterales,Solirubrobacteraceae,Solirubrobacter,NaN
4,ASV5,0.002820,0.005810,0.022699,0.005516,0.001006,0.004808,0.005975,0.000000,0.000000,...,0.002410,0.001629,0.001604,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Klebsiella,NaN


In [4]:
#извлекаю метки классов из названий столбцов
sample_labels = []
for col in df.columns[1:60]:  #пропускаю первый столбец (ASV) и таксономические данные
    if col.startswith("NG"):
        sample_labels.append("NG")
    elif col.startswith("NZ"):
        sample_labels.append("NZ")
    elif col.startswith("XZ"):
        sample_labels.append("XZ")

In [5]:
#преобразование в массив NumPy для удобства обработки
X = df.iloc[:, 1:60].T.values  #транспонирую, чтобы образцы соответствовали меткам
y = np.array(sample_labels)

In [6]:
#выбор 7 случайных образцов
np.random.seed(42)
unknown_indices = np.random.choice(len(y), 7, replace=False)
X_unknown = X[unknown_indices]
y_unknown = y[unknown_indices]

In [7]:
#удаление неизвестных образцов из обучающего/тестового набора
X_known = np.delete(X, unknown_indices, axis=0)
y_known = np.delete(y, unknown_indices)

In [8]:
#разделение на обучающий и тестовый наборы (80/20)
X_train, X_test, y_train, y_test = train_test_split(X_known, y_known, test_size=0.2, random_state=42)

In [9]:
#нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_unknown_scaled = scaler.transform(X_unknown)

In [10]:
#обучение модели Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

In [12]:
#предсказание среды для неизвестных образцов
y_unknown_pred = clf.predict(X_unknown_scaled)
print(y_unknown_pred)

['NG' 'NG' 'XZ' 'NZ' 'XZ' 'XZ' 'XZ']


In [14]:
#оценка модели
y_pred = clf.predict(X_test_scaled)
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

Classification Report:
              precision    recall  f1-score   support

          NG       1.00      1.00      1.00         2
          NZ       1.00      1.00      1.00         4
          XZ       1.00      1.00      1.00         5

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

